<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/07_RoBERTa_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Partisan-Responses-master')

Install all necessary libraries

In [ ]:
%%writefile setups.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setups.sh


In [ ]:
!sh setups.sh

In [ ]:
#!pip install -r requirements.txt
!pip install simpletransformers

Import all used libraries

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
from sklearn.model_selection import train_test_split

Read data

In [ ]:
speech = pd.read_pickle("search_dataset_small.pkl")

sp = speech['speech']
party = speech['party'].apply(lambda x: 1 if x == 'R' else 0)

df = pd.DataFrame({
    'text': sp,
    'labels': party
})
df=df.dropna(axis=0, how='any')
df=df[df.iloc[:,0].apply(lambda x: len(x.split())<1000)]
df=df.sample(frac=0.3, random_state=36)
train_df, eval_df = train_test_split(df, test_size=0.2)

Delete data to reduce RAM and VRAM footprint

In [ ]:
del speech
del sp
del party
del df

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
import gc
gc.collect(generation=2)

0

Train model

In [ ]:
model_args = ClassificationArgs(num_train_epochs=1, fp16=False, sliding_window=True, learning_rate=4e-6, save_eval_checkpoints= False,save_steps=50000)
model = ClassificationModel(
    "roberta", "roberta-base", args=model_args
)
model.train_model(train_df, output_dir='roberta')

Evaluate model

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


INFO:simpletransformers.classification.classification_model: 54743 features created from 54743 samples.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.3170334320525408, 'tp': 11802, 'tn': 24024, 'fp': 4622, 'fn': 14295, 'eval_loss': 0.614733768007219}


In [ ]:
result

{'eval_loss': 0.614733768007219,
 'fn': 14295,
 'fp': 4622,
 'mcc': 0.3170334320525408,
 'tn': 24024,
 'tp': 11802}